In [26]:
import os
import sys

sys.path.append('..')

import numpy as np
import pandas as pd
import matplotlib as plt
import sklearn
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
le = LabelEncoder()

df = pd.read_hdf("../data_extract/features_dataset_2b.h5")

In [27]:
NUM_PEOPLE = 40
df = df.loc[df['id']<NUM_PEOPLE]

In [28]:
from numpy import zeros
def gfit(chlist, xtrain, id_range):
    le = LabelEncoder()
    mm = list()
    covm = list()
    for pid in range(id_range):
        mm.append(np.mean(xtrain[chlist].loc[df['id']==pid],axis=0))
        covm.append(np.cov(xtrain[chlist].loc[df['id']==pid], rowvar=False))
    return mm, covm


def predict(chlist,xtest,mm,covm,y_test):
    cls = len(mm)
    total = y_test['id'].count()
    dist = zeros((total,cls))
    from scipy.stats import multivariate_normal
    for i in range(cls):
        dist[:,i] = multivariate_normal.pdf(xtest, mean=mm[i], cov=covm[i],allow_singular=True)
    le2 = LabelEncoder()
    le2.fit(y_test['fname'])
    totalfs = len(le2.classes_)
    true_results = 0
    for fname in le2.classes_:
        bar = y_test['fname']==fname
        probc = dist[bar,:]
        pidc = y_test.loc[y_test['fname']==fname]['id'].iloc[0]
        plist = np.prod(probc,axis=0)
        if plist[pidc] == max(plist):
            true_results = true_results + 1
    print(true_results,totalfs)
    print("Accuracy is {}%".format(true_results*100/totalfs))
    
def train_MLE(df):
    chlist = ["chroma{}".format(i) for i in range(12)]
    log_chroma_list = ["lc{}".format(i) for i in range(12)]
    df[log_chroma_list] = np.log(df[chlist])

    lclist = ["mfcc{}".format(i) for i in range(14)] + log_chroma_list
    
    from sklearn.model_selection import train_test_split
    x_train, x_test, y_train, y_test = train_test_split(
        df[lclist], df[["id","fname"]], test_size=0.3, random_state=42)
    
    ## Mean, standard deviation Scaling 
    scaler = preprocessing.StandardScaler().fit(x_train)
    x_train[lclist] = scaler.transform(x_train)
    x_test[lclist] = scaler.transform(x_test)
    
    x_train['id'] = y_train["id"]
    x_train['fname'] = y_train["fname"]
    
    mm, covm = gfit(lclist,x_train,NUM_PEOPLE)
    predict(lclist,x_test,mm,covm,y_test)

In [29]:
train_MLE(df)

(2471, 2703)
Accuracy is 91%
